In [1]:
import pandas as pd
import h5py
import os
import numpy as np

In [2]:
subset = 'val'
# tasks = ['animals', 'ghost', 'lego', 'talk']
tasks = ['talk']
data_path = f'/nas/project_data/B1_Behavior/hakim/UDIVA/annotation/{subset}'

In [3]:
train_session_path = '/nas/database/UDIVA/train/metadata/metadata_train/sessions_train.csv'
train_label_path = '/nas/database/UDIVA/train/metadata/metadata_train/parts_train.csv'
val_session_path = '/nas/database/UDIVA/val/metadata/metadata_val/sessions_val.csv'
val_label_path = '/nas/database/UDIVA/val/metadata/metadata_val_unmasked/parts_val_unmasked.csv'
test_session_path = '/nas/database/UDIVA/test/metadata/metadata_test/sessions_test.csv'
test_label_path = '/nas/database/UDIVA/test/metadata/metadata_test_unmasked/parts_test_unmasked.csv'

session_df = pd.read_csv(val_session_path, index_col='ID', usecols=['ID', 'PART.1', 'PART.2'])

label_df = pd.read_csv(val_label_path,
                       index_col='ID',
                       usecols=['ID', 
                                'OPENMINDEDNESS_Z', 
                                'CONSCIENTIOUSNESS_Z', 
                                'EXTRAVERSION_Z', 
                                'AGREEABLENESS_Z', 
                                'NEGATIVEEMOTIONALITY_Z'])

In [4]:
def get_personality(df, participant_id):
    personality = []
    personality.append(df.loc[participant_id]['OPENMINDEDNESS_Z'])
    personality.append(df.loc[participant_id]['CONSCIENTIOUSNESS_Z'])
    personality.append(df.loc[participant_id]['EXTRAVERSION_Z'])
    personality.append(df.loc[participant_id]['AGREEABLENESS_Z'])
    personality.append(df.loc[participant_id]['NEGATIVEEMOTIONALITY_Z'])
    return personality

In [5]:
def get_landmarks(task, session, participant, subset):
    data_path = f'/nas/project_data/B1_Behavior/hakim/UDIVA/annotation/{subset}'
    par_name = 'FC' + str(participant) + '_' + task[0].upper()
    if task == 'talk':
        annotation_path = os.path.join(data_path, task, session, par_name, 'annotations_cleaned_masked.hdf5')
    else:
        annotation_path = os.path.join(data_path, task, session, par_name, 'annotations_raw.hdf5')
    
    landmarks = []
    with h5py.File(annotation_path, "r") as f:
        for frame in f.keys():
            try:
                landmark = f[frame]['face']['landmarks'][()]
                landmarks.append(landmark)
            except KeyError:
                continue
        f.close()
    return np.array(landmarks)

In [6]:
for task in tasks:
    anno_path = os.path.join(data_path, task)
    
    for session in os.listdir(anno_path):
        session_path = os.path.join(anno_path, session)
        session_id = int(session)

        for par in range(1,3):
            if (par == 1):
                participant_id = session_df.loc[session_id]['PART.1']
            else:
                participant_id = session_df.loc[session_id]['PART.2']
            
            label = get_personality(label_df, participant_id)
            landmarks = get_landmarks(task, session, par, subset)
        break   
    break

In [6]:
l = get_landmarks("animals", "008105", 1, 'test')

In [8]:
l.shape

(9486, 68, 3)

In [9]:
m = get_landmarks("animals", "008105", 2, 'test')

In [10]:
m.shape

(9490, 68, 3)